In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,41770
2,Huelva,Confirmados PDIA 14 días,3039
3,Huelva,Tasa PDIA 14 días,"592,2014147358575"
4,Huelva,Confirmados PDIA 7 días,1447
5,Huelva,Tasa PDIA 7 dias,"281,9728355125982"
6,Huelva,Total Confirmados,41968
7,Huelva,Curados,35238
8,Huelva,Fallecidos,396


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  41770.0


/tmp/ipykernel_4167/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  11666.0


/tmp/ipykernel_4167/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_4167/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_4167/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_4167/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 331 personas en los últimos 7 días 

Un positivo PCR cada 171 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,41770.0,1447.0,3039.0,513170.0,281.972836,592.201415,273.0
Huelva-Costa,24546.0,855.0,1766.0,289548.0,295.287828,609.916145,159.0
Condado-Campiña,13131.0,487.0,1018.0,156231.0,311.717905,651.599234,98.0
Huelva (capital),11666.0,434.0,840.0,143837.0,301.730431,583.994383,72.0
Lepe,2750.0,105.0,209.0,27880.0,376.614060,749.641320,27.0
Bollullos Par del Condado,1204.0,61.0,95.0,14387.0,423.993883,660.318343,21.0
Almonte,2140.0,41.0,94.0,24507.0,167.299139,383.563880,17.0
Sierra de Huelva-Andévalo Central,3708.0,101.0,247.0,67391.0,149.871645,366.517784,16.0
Ayamonte,1797.0,82.0,155.0,21104.0,388.551933,734.457923,14.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Sanlúcar de Guadiana,30.0,11.0,23.0,403.0,2729.528536,5707.196030,0.0
Marines (Los),14.0,4.0,8.0,399.0,1002.506266,2005.012531,0.0
Corteconcepción,43.0,5.0,9.0,536.0,932.835821,1679.104478,1.0
Escacena del Campo,203.0,18.0,37.0,2287.0,787.057280,1617.839965,2.0
Cumbres Mayores,120.0,3.0,28.0,1749.0,171.526587,1600.914808,0.0
Cañaveral de León,15.0,2.0,6.0,396.0,505.050505,1515.151515,0.0
Villarrasa,213.0,19.0,27.0,2211.0,859.339665,1221.166893,3.0
Villablanca,252.0,12.0,35.0,2885.0,415.944541,1213.171577,5.0
Nava (La),6.0,1.0,3.0,258.0,387.596899,1162.790698,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Cumbres Mayores,120.0,3.0,28.0,1749.0,171.526587,1600.914808,0.0,0.107143
Villanueva de los Castillejos,190.0,1.0,9.0,2825.0,35.398230,318.584071,0.0,0.111111
Encinasola,161.0,1.0,8.0,1310.0,76.335878,610.687023,0.0,0.125000
Cerro de Andévalo (El),185.0,2.0,8.0,2327.0,85.947572,343.790288,0.0,0.250000
Alosno,221.0,2.0,8.0,3933.0,50.851767,203.407068,0.0,0.250000
Aroche,107.0,1.0,4.0,3054.0,32.743942,130.975769,0.0,0.250000
Manzanilla,132.0,4.0,15.0,2118.0,188.857413,708.215297,0.0,0.266667
Gibraleón,869.0,21.0,78.0,12737.0,164.873989,612.389103,6.0,0.269231
Calañas,239.0,4.0,14.0,2768.0,144.508671,505.780347,0.0,0.285714
